[X] Caffe2  
[] TF  
[] CNTK  
[] MXNet    

In [18]:
# Keras Model


import onnxmltools
import json
import numpy as np
from keras import Sequential
from keras.layers import Dense


def f1(**kwargs):
    params_json = json.load(open('../imly/architectures/sklearn/params.json'))
    params = params_json['params']
    kwargs.setdefault('params', params)
    kwargs.setdefault('x_train', np.array([[1], [2]]))

    model = Sequential()
    model.add(Dense(kwargs['params']['first_neuron'],
                    input_dim=kwargs['x_train'].shape[1],
                    activation=kwargs['params']['activation']))

    model.compile(optimizer=kwargs['params']['optimizer'],
                  loss=kwargs['params']['losses'],
                  metrics=['acc'])
    onnx_model = onnxmltools.convert_keras(model, target_opset=8)
    print(type(model))
    onnx_model
    return onnx_model


def f2():

    model = Sequential()
    model.add(Dense(1,
                    input_dim=4,
                    activation='linear'))

    model.compile(optimizer='adam',
                  loss='mse',
                  metrics=['mse'])
    onnx_model = onnxmltools.convert_keras(model, target_opset=8)
    print(type(model))
    onnx_model
    return onnx_model

In [19]:
onnx_model = f2()

The maximum opset needed by this model is only 7.


<class 'keras.engine.sequential.Sequential'>


In [20]:
onnx_model

ir_version: 3
producer_name: "OnnxMLTools"
producer_version: "1.3.1"
domain: "onnxml"
model_version: 0
doc_string: ""
graph {
  node {
    input: "dense_3_input_0"
    input: "W"
    output: "transformed_tensor"
    name: "_class__keras_layers_core_Dense__"
    op_type: "MatMul"
    domain: ""
  }
  node {
    input: "transformed_tensor"
    input: "B"
    output: "biased_tensor_name"
    name: "Add"
    op_type: "Add"
    domain: ""
  }
  node {
    input: "biased_tensor_name"
    output: "dense_3_BiasAdd_01"
    name: "Identity"
    op_type: "Identity"
    domain: ""
  }
  name: "e4e3a80ce8544214b5c1665a274d2627"
  initializer {
    dims: 4
    dims: 1
    data_type: FLOAT
    float_data: -0.5307927131652832
    float_data: 0.7320890426635742
    float_data: 0.7663308382034302
    float_data: 0.867125391960144
    name: "W"
  }
  initializer {
    dims: 1
    data_type: FLOAT
    float_data: 0.0
    name: "B"
  }
  input {
    name: "dense_3_input_0"
    type {
      tensor_type {
  

In [21]:
import onnx
onnxmltools.utils.save_model(onnx_model, '../data/model.onnx')

---

### TensorFlow ###

In [22]:
import onnx
import warnings
from onnx_tf.backend import prepare

In [23]:
warnings.filterwarnings('ignore') # Ignore all the warning messages in this tutorial
model = onnx.load('../data/model.onnx') # Load the ONNX file
tf_rep = prepare(model) # Import the ONNX model to Tensorflow

ValueError: '_class__keras_layers_core_Dense__' is not a valid scope name

In [17]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import sys
import onnx
import tensorflow
import onnx_tf
import pkg_resources

print("Python version:")
print(sys.version)
print("ONNX version:")
print(onnx.version.version)
print("ONNX-TF version:")
print(pkg_resources.get_distribution("onnx-tf").version)
print("Tensorflow version:")
print(tensorflow.__version__)

Python version:
3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:27:44) [MSC v.1900 64 bit (AMD64)]
ONNX version:
1.3.0
ONNX-TF version:
1.2.1
Tensorflow version:
1.12.0


---

#### Notes ####
1. Unable to do Keras -> ONNX -> TF
    + Possibly because only TF -> ONNX -> TF is possible
    + Or only limited Keras models can be transformed to TF
    + Possible solutions
        + Check if these errors are caused by ONNX file configs
        + Check at TF level
        + If nether works, check TF -> ONNX -> TF flow